#marker&compositional analysis

In [ ]:
import scanpy as sc, numpy as np, pandas as pd
import seaborn as sns
from scipy.stats import median_abs_deviation
import scvi

In [ ]:
scvi.settings.seed = 0

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
from matplotlib.pyplot import rc_context
from matplotlib import pyplot as plt

In [ ]:
import os
os.environ["R_HOME"] = "/data/User/revolvefire/miniforge3/envs/rpy_base_240331/lib/R"
os.environ['LD_LIBRARY_PATH'] = "/data/User/revolvefire/miniforge3/envs/rpy_base_240331/lib/R/bin"

In [ ]:
import anndata2ri
import logging

In [ ]:
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
print(ro.r(".libPaths()"))

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

In [ ]:
mainDir = "/mnt/gmi-l1/_90.User_Data/revolvefire/fastq/PPP_GMI"

In [ ]:
objectDir = "/mnt/gmi-l1/_90.User_Data/revolvefire/projects/3.PPP_231118/231128_PPP/scobject/"

In [ ]:
adata = sc.read(objectDir+ "/241216.2000count.5ribo.scVIintegrated.fullgene.melanocyte.label2.recolor.h5ad")

In [ ]:
adata

In [ ]:
sc.set_figure_params(scanpy=True, dpi=150, dpi_save=300)

In [ ]:
sc.pl.umap(
    adata,
    color=["label2"], frameon=False, add_outline=True, size=10, legend_fontsize="x-small", save="241216.umap.svg",)

In [ ]:
# Plot the UMAP with custom colors
sc.pl.umap(
    adata,
    color="bulk_ident",
    frameon=False,
    add_outline=True,
    size=20, save="241216.bulk.umap.svg", title=None
)

In [ ]:
sc.set_figure_params(scanpy=True, dpi=150, dpi_save=300)

In [ ]:
markers=['PTTG1', 'FOXM1', 'TP63', 'ITGB4', 'KRT5', 'KRT14', 'KRT1', 'KRT10', 'TGM1', 'IVL','SPRR2G', 'SPINK5', 'KRT6A', 'KRT16', 'CD68', 'HLA-DRA', 'CD3D', 'CD3E', 'TYRP1', 'MLANA', 'PECAM1', 'CLDN5', 'TAGLN', 'ACTA2', 'PDGFRA', 'LUM']
sc.pl.dotplot(adata, markers, groupby='label2', standard_scale="var", cmap="Spectral_r", show=False)

fig = plt.gcf()     
ax = fig.axes[0]  

tick_labels = [tick.get_text() for tick in ax.get_xticklabels()]
italic_labels = [f"$\\mathit{{{label}}}$" for label in tick_labels]
tick_positions = np.arange(len(tick_labels)) + 0.5  

ax.set_xticks(tick_positions)
ax.set_xticklabels(italic_labels)

fig.savefig("./figure1/250311.dotplot.svg", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pertpy as pt
plt.rcParams["figure.figsize"] = (7, 7)
sc.settings.verbosity = 3

In [ ]:
## Initialize object for Milo analysis
milo = pt.tl.Milo()
mdata = milo.load(adata)

In [ ]:
mdata

In [ ]:
sc.pp.neighbors(mdata["rna"], use_rep="X_scVI", n_neighbors=30)

In [ ]:
milo.make_nhoods(mdata["rna"], prop=1) 

In [ ]:
mdata["rna"].obsm["nhoods"]

In [ ]:
mdata["rna"][mdata["rna"].obs["nhood_ixs_refined"] != 0].obs[["nhood_ixs_refined", "nhood_kth_distance"]]

In [ ]:
nhood_size = np.array(mdata["rna"].obsm["nhoods"].sum(0)).ravel()
plt.hist(nhood_size, bins=100)
plt.xlabel("# cells in nhood")
plt.ylabel("# nhoods");

In [ ]:
mdata = milo.count_nhoods(mdata, sample_col="orig_ident")

In [ ]:
mdata

In [ ]:
mdata["milo"]

In [ ]:
# Reorder categories
# (by default, the last category is taken as the condition of interest)
mdata["rna"].obs["bulk_ident2"] = mdata["rna"].obs["bulk_ident2"].cat.reorder_categories(["HC", "Peri-lesion", "Lesion"])

In [ ]:
milo.da_nhoods(mdata, design="~bulk_ident")

In [ ]:
mdata["milo"]

In [ ]:
mean_n_cells = mdata["milo"].X.toarray().mean(0)
plt.plot(nhood_size, mean_n_cells, ".")
plt.xlabel("# cells in nhood")
plt.ylabel("Mean # cells per sample in nhood")

In [ ]:
mdata["milo"].obs

In [ ]:
mdata["milo"].var

In [ ]:
old_figsize = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = [10, 5]
plt.subplot(1, 2, 1)
plt.hist(mdata["milo"].var.PValue, bins=50)
plt.xlabel("P-Vals")
plt.subplot(1, 2, 2)
plt.plot(mdata["milo"].var.logFC, -np.log10(mdata["milo"].var.SpatialFDR), ".")
plt.xlabel("log-Fold Change")
plt.ylabel("- log10(Spatial FDR)")
plt.tight_layout()
plt.rcParams["figure.figsize"] = old_figsize

In [ ]:
milo.build_nhood_graph(mdata)

In [ ]:
plt.rcParams["figure.figsize"] = [4,4]
milo.plot_nhood_graph(
    mdata,
    alpha=0.2,  
    min_size=1,  
    plot_edges=False
)

In [ ]:
milo.annotate_nhoods(mdata, anno_col="label2")

In [ ]:
plt.hist(mdata["milo"].var["nhood_annotation_frac"], bins=30)
plt.xlabel("celltype fraction")

In [ ]:
import pandas

In [ ]:
# Add the new category to the existing categories
mdata["milo"].var["nhood_annotation"] = mdata["milo"].var["nhood_annotation"].cat.add_categories("Mixed")

In [ ]:
mdata["milo"].var.loc[mdata["milo"].var["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
milo.plot_da_beeswarm(mdata, alpha=0.2)

In [ ]:
color = {
    "BK-1": "#1f77b4",
    "BK-2": "#17becf",
    "BK-3": "#279e68",
    "SBK-1": "#b5bd61",
    "SBK-2": "#aa40fc",
    "SBK-3": "#ff9896",
    "SBK-4": "#e377c2",
    "SBK-5": "#d62728",
    "Myeloid cell": "#ffbb78",
    "Lymphocyte": "#aec7e8",
    "Melanocyte": "#ff7f0e",
    "Endothelial": "#98df8a",
    "Pericyte": "#8c564b",
    "Fibroblast": "#c5b0d5",
    'Mixed': '#aeaeae',
}

In [ ]:
plt.rcParams["figure.figsize"] = (6, 6)

In [ ]:
# Assuming you have a list of unique categories in the 'nhood_annotation' column
unique_categories = mdata["milo"].var["nhood_annotation"].unique()

# Define a color palette (using seaborn or manually)
palette = sns.color_palette("hsv", len(unique_categories))  # or any other palette

# Map the palette to the unique categories
#color_mapping = dict(zip(unique_categories, palette))

# Now, pass the color mapping to the plot function
ax = milo.plot_da_beeswarm(mdata, alpha=0.2, palette=color)
ax.grid(False)  # Disable the grid on the current axis

fig = ax.get_figure()  # Access the figure from the axis
fig.savefig("241217.milo.svg", dpi=300, bbox_inches="tight")

plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = [3, 6]

In [ ]:
adata = sc.read(objectDir+ "/250311.2000count.5ribo.scVIintegrated.fullgene.melanocyte.label2.recolor.pustular.h5ad")

In [ ]:
import pandas as pd
from matplotlib.pyplot import rc_context
from matplotlib import pyplot as plt

tmp = pd.crosstab(adata.obs['bulk_ident2'], adata.obs['label2'], normalize='index')

column_order = [
    "BK-1", "BK-2", "BK-3", "SBK-1", "SBK-2", "SBK-3", "SBK-4", "SBK-5",
    "Fibroblast", "Pericyte", "Endothelial", "Melanocyte",
    "Lymphocyte", "Myeloid cell"
]
tmp = tmp[column_order]

counts = adata.obs['bulk_ident2'].value_counts().reindex(tmp.index)

colors = {
    "BK-1": "#1f77b4",
    "BK-2": "#17becf",
    "BK-3": "#279e68",
    "SBK-1": "#b5bd61",
    "SBK-2": "#aa40fc",
    "SBK-3": "#ff9896",
    "SBK-4": "#e377c2",
    "SBK-5": "#d62728",
    "Myeloid cell": "#ffbb78",
    "Lymphocyte": "#aec7e8",
    "Melanocyte": "#ff7f0e",
    "Endothelial": "#98df8a",
    "Pericyte": "#8c564b",
    "Fibroblast": "#c5b0d5"
}

with rc_context():
    fig, ax = plt.subplots(figsize=(3, 6))
    tmp.plot.bar(stacked=True, color=colors, ax=ax)

    for container in ax.containers:
        for patch in container:
            patch.set_edgecolor('black')
            patch.set_linewidth(0.5)

    ax.set_xlabel('')

    plt.xticks(rotation=0)
    
    xticks = ax.get_xticks()
    for i, group in enumerate(tmp.index):
        total = counts.loc[group]
        ax.text(xticks[i], 1.02, f"N={total}", ha='center', va='bottom', transform=ax.get_xaxis_transform())

    plt.grid(False)
    plt.ylim(0, 1)
    plt.legend(loc='upper right', bbox_to_anchor=(1.7, 1.02), reverse=True)
    plt.savefig('250403.barplot.svg', dpi=300, bbox_inches='tight')
    plt.show()